In [1]:
import torch, os, copy, time
from tqdm import tqdm
from torch import nn
from torch.utils.data import DataLoader, Dataset
from utils import *
import torch.nn.functional as F

Epoch: 1 | Batch index: 0 | Batch size: 128
break minibatch for-loop
Epoch: 2 | Batch index: 0 | Batch size: 128
break minibatch for-loop


In [2]:
class logReg(torch.nn.Module):
    # logistic regression 
    def __init__(self, num_features, num_classes):
        super().__init__()
        self.num_classes = num_classes
        self.linear = torch.nn.Linear(num_features, num_classes)

    def forward(self, x):
        logits = self.linear(x.float())
        probas = torch.sigmoid(logits)
        return probas.type(torch.FloatTensor)

global_model = logReg(num_features=NUM_FEATURES, num_classes=2)
global_weights = global_model.state_dict()

In [3]:
class DatasetSplit(Dataset):
    """An abstract Dataset class wrapped around Pytorch Dataset class.
    """

    def __init__(self, dataset, idxs):
        self.dataset = dataset
        self.idxs = [int(i) for i in idxs]

    def __len__(self):
        return len(self.idxs)

    def __getitem__(self, item):
        feature, label, sensitive = self.dataset[self.idxs[item]]
        return feature, label, sensitive


class LocalUpdate(object):
    def __init__(self, dataset, idxs, loss_func = None, fairness_coefficient = 2):
        self.trainloader, self.validloader, self.testloader = self.train_val_test(dataset, list(idxs))
        # Default criterion set to NLL loss function
        self.criterion = nn.NLLLoss().to(DEVICE)

    def train_val_test(self, dataset, idxs):
        """
        Returns train, validation and test dataloaders for a given dataset
        and user indexes.
        """
        # split indexes for train, validation, and test (80, 10, 10)
        idxs_train = idxs[:int(0.8*len(idxs))]
        idxs_val = idxs[int(0.8*len(idxs)):int(0.9*len(idxs))]
        idxs_test = idxs[int(0.9*len(idxs)):]

        trainloader = DataLoader(DatasetSplit(dataset, idxs_train),
                                 batch_size=BATCH_SIZE, shuffle=True)
        validloader = DataLoader(DatasetSplit(dataset, idxs_val),
                                 batch_size=int(len(idxs_val)/10), shuffle=False)
        testloader = DataLoader(DatasetSplit(dataset, idxs_test),
                                batch_size=int(len(idxs_test)/10), shuffle=False)
        return trainloader, validloader, testloader

    def update_weights(self, model, global_round):
        # Set mode to train model
        model.train()
        epoch_loss = []

        # Set optimizer for the local updates
        if OPTIMIZER == 'sgd':
            optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE,
                                        momentum=0.5)
        elif OPTIMIZER == 'adam':
            optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE,
                                         weight_decay=1e-4)

        for iter in range(LOCAL_EPOCHS):
            batch_loss = []
            for batch_idx, (features, labels, _) in enumerate(self.trainloader):
                features, labels = features.to(DEVICE), labels.to(DEVICE).type(torch.LongTensor)
                # we need to set the gradients to zero before starting to do backpropragation 
                # because PyTorch accumulates the gradients on subsequent backward passes. 
                # This is convenient while training RNNs
                model.zero_grad()
                
                log_probs = model(features)
                loss = self.criterion(log_probs, labels)
                loss.backward()
                optimizer.step()

                if batch_idx % 50 == 0:
                    print('| Global Round : {} | Local Epoch : {} | [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                        global_round, iter, batch_idx * len(features),
                        len(self.trainloader.dataset),
                        100. * batch_idx / len(self.trainloader), loss.item()))
                batch_loss.append(loss.item())
            epoch_loss.append(sum(batch_loss)/len(batch_loss))

        # weight, loss
        return model.state_dict(), sum(epoch_loss) / len(epoch_loss)

    def inference(self, model):
        """ 
        Returns the inference accuracy, 
                                loss, 
                                N(sensitive group, pos), 
                                N(non-sensitive group, pos), 
                                N(sensitive group),
                                N(non-sensitive group)
        """

        model.eval()
        loss, total, correct = 0.0, 0.0, 0.0
        sp, nsp, s, n = 0, 0, 0, 0
        for batch_idx, (features, labels, sensitive) in enumerate(self.testloader):
            features, labels = features.to(DEVICE), labels.to(DEVICE).type(torch.LongTensor)
            sensitive = sensitive.to(DEVICE)
            
            # Inference
            outputs = model(features)

            # Prediction
            _, pred_labels = torch.max(outputs, 1)
            pred_labels = pred_labels.view(-1)
            bool_correct = torch.eq(pred_labels, labels)
            correct += torch.sum(bool_correct).item()
            total += len(labels)
            
            bool_sensitive = torch.eq(sensitive, torch.ones(len(labels)))
            s += torch.sum(bool_sensitive).item()
            n += torch.sum(torch.logical_not(bool_sensitive)).item()
            sp += torch.sum(torch.logical_and(bool_correct, bool_sensitive)).item()
            nsp += torch.sum(torch.logical_and(bool_correct, torch.logical_not(bool_sensitive))).item()
            
            batch_loss = self.criterion(outputs, labels)
            loss += batch_loss.item()

        accuracy = correct/total
        return accuracy, loss, s, n, sp, nsp


def test_inference(model, test_dataset):
    """ Returns the test accuracy and loss.
    """

    model.eval()
    loss, total, correct = 0.0, 0.0, 0.0
    sp, nsp, s, n = 0, 0, 0, 0
    
    criterion = nn.NLLLoss().to(DEVICE)
    testloader = DataLoader(test_dataset, batch_size=128,
                            shuffle=False)

    for batch_idx, (features, labels, sensitive) in enumerate(testloader):
        features = features.to(DEVICE)
        labels =  labels.to(DEVICE).type(torch.LongTensor)
        # Inference
        outputs = model(features)
        batch_loss = criterion(outputs, labels)
        loss += batch_loss.item()

        # Prediction
        _, pred_labels = torch.max(outputs, 1)
        pred_labels = pred_labels.view(-1)
        bool_correct = torch.eq(pred_labels, labels)
        correct += torch.sum(bool_correct).item()
        total += len(labels)

        bool_sensitive = torch.eq(sensitive, torch.ones(len(labels)))
        s += torch.sum(bool_sensitive).item()
        n += torch.sum(torch.logical_not(bool_sensitive)).item()
        sp += torch.sum(torch.logical_and(bool_correct, bool_sensitive)).item()
        nsp += torch.sum(torch.logical_and(bool_correct, torch.logical_not(bool_sensitive))).item()

    accuracy = correct/total
    return accuracy, loss, s, n, sp, nsp

In [4]:
# Training
train_loss, train_accuracy = [], []
val_acc_list, net_list = [], []
cv_loss, cv_acc = [], []
print_every = 2
val_loss_pre, counter = 0, 0
start_time = time.time()

def average_weights(w):
    """
    Returns the average of the weights.
    """
    w_avg = copy.deepcopy(w[0])
    for key in w_avg.keys():
        for i in range(1, len(w)):
            w_avg[key] += w[i][key]
        w_avg[key] = torch.div(w_avg[key], len(w))
    return w_avg

for epoch in tqdm(range(NUM_EPOCHS)):
    local_weights, local_losses = [], []
    print(f'\n | Global Training Round : {epoch+1} |\n')

    global_model.train()
    m = max(int(FRAC * NUM_CLIENTS), 1) # the number of clients to be chosen in each epoch
    idxs_users = np.random.choice(range(NUM_CLIENTS), m, replace=False)

    for idx in idxs_users:
        local_model = LocalUpdate(dataset=train_dataset,
                                    idxs=clients_idx[idx])
        w, loss = local_model.update_weights(
            model=copy.deepcopy(global_model), global_round=epoch)
        local_weights.append(copy.deepcopy(w))
        local_losses.append(copy.deepcopy(loss))

    # update global weights
    global_weights = average_weights(local_weights)
    global_model.load_state_dict(global_weights)

    loss_avg = sum(local_losses) / len(local_losses)
    train_loss.append(loss_avg)

    # Calculate avg training accuracy over all users at every epoch
    list_acc, list_loss = [], []
    s, n, sp, nsp = 0, 0, 0, 0
    global_model.eval()
    for c in range(NUM_CLIENTS):
        local_model = LocalUpdate(dataset=train_dataset,
                                    idxs=clients_idx[idx])
        acc, loss, s_, n_, sp_, nsp_ = local_model.inference(model=global_model)
        list_acc.append(acc)
        list_loss.append(loss)
        s += s_
        n += n_
        sp += sp_
        nsp += nsp_
    train_accuracy.append(sum(list_acc)/len(list_acc))

    # print global training loss after every 'i' rounds
    if (epoch+1) % print_every == 0:
        print(f' \nAvg Training Stats after {epoch+1} global rounds:')
        print(f'Training Loss : {np.mean(np.array(train_loss))}')
        print('Train Accuracy: {:.2f}%'.format(100*train_accuracy[-1]))
        print('Train Risk Difference: {:.2f} \n'.format(abs(sp/s-nsp/n)))

# Test inference after completion of training
test_acc, test_loss, test_s, test_n, test_sp, test_nsp = test_inference(global_model, test_dataset)

print(f' \n Results after {NUM_EPOCHS} global rounds of training:')
print("|---- Avg Train Accuracy: {:.2f}%".format(100*train_accuracy[-1]))
print("|---- Test Accuracy: {:.2f}%".format(100*test_acc))

# Compute RD: risk difference - fairness metric
# |P(Group1, pos) - P(Group2, pos)| = |N(Group1, pos)/N(Group1) - N(Group2, pos)/N(Group2)|
print("|---- Test RD: {:.2f}".format(abs(test_sp/test_s - test_nsp/test_n)))

print('\n Total Run Time: {0:0.4f}'.format(time.time()-start_time))

  0%|          | 0/10 [00:00<?, ?it/s]


 | Global Training Round : 1 |

| Global Round : 0 | Local Epoch : 0 | [0/9945 (0%)]	Loss: -0.495596
| Global Round : 0 | Local Epoch : 0 | [6400/9945 (64%)]	Loss: -0.999537
| Global Round : 0 | Local Epoch : 1 | [0/9945 (0%)]	Loss: -0.999529
| Global Round : 0 | Local Epoch : 1 | [6400/9945 (64%)]	Loss: -0.999475
| Global Round : 0 | Local Epoch : 2 | [0/9945 (0%)]	Loss: -0.999440
| Global Round : 0 | Local Epoch : 2 | [6400/9945 (64%)]	Loss: -0.999421
| Global Round : 0 | Local Epoch : 3 | [0/9945 (0%)]	Loss: -0.999442
| Global Round : 0 | Local Epoch : 3 | [6400/9945 (64%)]	Loss: -0.999484
| Global Round : 0 | Local Epoch : 4 | [0/9945 (0%)]	Loss: -0.999480
| Global Round : 0 | Local Epoch : 4 | [6400/9945 (64%)]	Loss: -0.999522
| Global Round : 0 | Local Epoch : 5 | [0/9945 (0%)]	Loss: -0.999487
| Global Round : 0 | Local Epoch : 5 | [6400/9945 (64%)]	Loss: -0.999462
| Global Round : 0 | Local Epoch : 6 | [0/9945 (0%)]	Loss: -0.999580
| Global Round : 0 | Local Epoch : 6 | [6400/9

 10%|█         | 1/10 [00:07<01:07,  7.54s/it]

| Global Round : 0 | Local Epoch : 9 | [12800/16103 (79%)]	Loss: -0.999703

 | Global Training Round : 2 |

| Global Round : 1 | Local Epoch : 0 | [0/9945 (0%)]	Loss: -0.999406
| Global Round : 1 | Local Epoch : 0 | [6400/9945 (64%)]	Loss: -0.999539
| Global Round : 1 | Local Epoch : 1 | [0/9945 (0%)]	Loss: -0.999699
| Global Round : 1 | Local Epoch : 1 | [6400/9945 (64%)]	Loss: -0.999646
| Global Round : 1 | Local Epoch : 2 | [0/9945 (0%)]	Loss: -0.999645
| Global Round : 1 | Local Epoch : 2 | [6400/9945 (64%)]	Loss: -0.999595
| Global Round : 1 | Local Epoch : 3 | [0/9945 (0%)]	Loss: -0.999684
| Global Round : 1 | Local Epoch : 3 | [6400/9945 (64%)]	Loss: -0.999670
| Global Round : 1 | Local Epoch : 4 | [0/9945 (0%)]	Loss: -0.999620
| Global Round : 1 | Local Epoch : 4 | [6400/9945 (64%)]	Loss: -0.999577
| Global Round : 1 | Local Epoch : 5 | [0/9945 (0%)]	Loss: -0.999601
| Global Round : 1 | Local Epoch : 5 | [6400/9945 (64%)]	Loss: -0.999653
| Global Round : 1 | Local Epoch : 6 | [

 20%|██        | 2/10 [00:14<00:59,  7.45s/it]

...

 90%|█████████ | 9/10 [01:06<00:07,  7.36s/it]

| Global Round : 8 | Local Epoch : 9 | [12800/16103 (79%)]	Loss: -0.999727

 | Global Training Round : 10 |

| Global Round : 9 | Local Epoch : 0 | [0/9945 (0%)]	Loss: -0.999194
| Global Round : 9 | Local Epoch : 0 | [6400/9945 (64%)]	Loss: -0.999471
| Global Round : 9 | Local Epoch : 1 | [0/9945 (0%)]	Loss: -0.999621
| Global Round : 9 | Local Epoch : 1 | [6400/9945 (64%)]	Loss: -0.999601
| Global Round : 9 | Local Epoch : 2 | [0/9945 (0%)]	Loss: -0.999707
| Global Round : 9 | Local Epoch : 2 | [6400/9945 (64%)]	Loss: -0.999656
| Global Round : 9 | Local Epoch : 3 | [0/9945 (0%)]	Loss: -0.999640
| Global Round : 9 | Local Epoch : 3 | [6400/9945 (64%)]	Loss: -0.999695
| Global Round : 9 | Local Epoch : 4 | [0/9945 (0%)]	Loss: -0.999666
| Global Round : 9 | Local Epoch : 4 | [6400/9945 (64%)]	Loss: -0.999660
| Global Round : 9 | Local Epoch : 5 | [0/9945 (0%)]	Loss: -0.999662
| Global Round : 9 | Local Epoch : 5 | [6400/9945 (64%)]	Loss: -0.999721
| Global Round : 9 | Local Epoch : 6 | 

100%|██████████| 10/10 [01:13<00:00,  7.37s/it]

| Global Round : 9 | Local Epoch : 9 | [12800/16103 (79%)]	Loss: -0.999784
 
Avg Training Stats after 10 global rounds:
Training Loss : -0.999476874977605
Train Accuracy: 71.29%
Train Risk Difference: 0.19 



 
 Results after 10 global rounds of training:
|---- Avg Train Accuracy: 71.29%
|---- Test Accuracy: 76.38%
|---- Test RD: 0.19

 Total Run Time: 74.0647
